In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
pd.options.display.max_columns=None

import tensorflow as tf

In [207]:
pos = pd.read_csv('data/players_positions.csv')
games = pd.read_csv('data/games.csv')
clubs = pd.read_csv('data/clubs.csv')

GK_stats = pd.read_csv('data/raw_from_scrape/sofifa_gk_stats.csv')
stats = pd.read_csv('data/raw_from_scrape/sofifa_stats.csv')
DCU_games = pd.read_csv('data/DCU_games.csv')#, usecols=['Date', 'HomeTeam', 'AwayTeam', 'HomeTeam_id', 'AwayTeam_id', 'FTHG', 'FTAG', 'HY', 'AY', 'HR', 'AR'])
DCU_games.Date = pd.to_datetime(DCU_games.Date, format = "%Y-%m-%d")

In [122]:
display(players_pos.head())
display(games.head())

,journee,game_id,club_id,player_id,sofifa_id,poste,horizontal,vertical,field_poste
0,22,515694,22,33459,199482.0,Gar.,49.94,1.00,Gar.
1,22,515694,22,25928,179789.0,Déf.,22.12,11.38,Déf.
2,22,515694,22,51548,216643.0,Déf.,12.30,19.73,Déf.
3,22,515694,22,73302,217821.0,Déf.,87.60,19.73,Déf.
4,22,515694,22,83190,262138.0,Déf.,77.75,11.38,Déf.


,journee,game_id,home_club_id,away_club_id,home_goals,away_goals
0,18,515653,15,9,3,2
1,18,515650,267,17,0,4
2,18,515655,211,38,2,0
3,18,515651,43,22,0,0
4,18,515656,14,46,1,2


In [171]:
np.concatenate((np.asarray([3,4]), np.asarray([5,6]))  )

array([3, 4, 5, 6])

In [72]:
def create_AVG_stats_array(club_id):

    GK = GK_stats.loc[GK_stats.sofifa_id.isin(pos.loc[(pos.club_id == club_id) & (pos.field_poste == 'Gar.'),'sofifa_id'].values),:].mean()[1:]
    GK = GK/100 #Normalization
    GK = np.append(np.asarray([1]), GK) #addition of the number of GK during the match. Relicat suboptimal et pour tout dire superflu du modèle d'entrainement...

    ATK = stats.loc[stats.sofifa_id.isin(pos.loc[(pos.club_id == club_id) & (pos.field_poste == 'Att.'),'sofifa_id'].values),:]
    np_ATK = ATK.shape[0]
    ATK = ATK.mean()[1:]/100
    ATK[2:5] = ATK[2:5]*20
    ATK = ATK.values
    ATK = np.append(np.asarray([np_ATK]), ATK)

    MID = stats.loc[stats.sofifa_id.isin(pos.loc[(pos.club_id == club_id) & (pos.field_poste == 'Mil.'),'sofifa_id'].values),:]
    np_MID = MID.shape[0]
    MID = MID.mean()[1:]/100
    MID[2:5] = MID[2:5]*20
    MID = MID.values
    MID = np.append(np.asarray([np_MID]), MID)

    DEF = stats.loc[stats.sofifa_id.isin(pos.loc[(pos.club_id == club_id) & (pos.field_poste == 'Déf.'),'sofifa_id'].values),:]
    np_DEF = DEF.shape[0]
    DEF = DEF.mean()[1:]/100
    DEF[2:5] = DEF[2:5]*20
    DEF = DEF.values
    DEF = np.append(np.asarray([np_DEF]), DEF)
    return np.concatenate((GK,ATK,MID,DEF))

In [84]:
def create_MAX_stats_array(club_id):

    ATK = stats.loc[stats.sofifa_id.isin(pos.loc[(pos.club_id == club_id) & (pos.field_poste == 'Att.'),'sofifa_id'].values),:]
    ATK = ATK.max()[1:-1]/100
    ATK[2:5] = ATK[2:5]*20
    ATK = ATK.values

    MID = stats.loc[stats.sofifa_id.isin(pos.loc[(pos.club_id == club_id) & (pos.field_poste == 'Mil.'),'sofifa_id'].values),:]
    MID = MID.max()[1:-1]/100
    MID[2:5] = MID[2:5]*20
    MID = MID.values

    DEF = stats.loc[stats.sofifa_id.isin(pos.loc[(pos.club_id == club_id) & (pos.field_poste == 'Déf.'),'sofifa_id'].values),:]
    DEF = DEF.max()[1:-1]/100
    DEF[2:5] = DEF[2:5]*20
    DEF = DEF.values
    return np.concatenate((ATK,MID,DEF))

In [149]:
def create_club_history(club_id):
    # Attention changer le [1:11, :] en [0:10, :] pour faire des prédiction
    last_10_games = DCU_games.loc[(DCU_games.HomeTeam_id == club_id) | (DCU_games.AwayTeam_id == club_id),:]\
        .sort_values('Date', ascending = False).reset_index(drop = True).loc[1:11,:].drop(columns=['Date'])
    # voici les colonnes utilisées dans le modèle
    # ['goals', 'goals_conceded', 'HorA', 'yellow_cards', 'red_cards', 'exists']
    # HorA indique si l'équipe a joué match à domicile (HorA = 0) ou en tant que visiteur (HorA = 1). C'est un relicat obsolète des tentatives de modèle avec LSTM... A SUPPRIMER QUAND ON FERA UN MODELE PROPRE
    # exists = 1 indique qu'il y a au moins une partie dans l'hisorique. Sinon exists = 0 #####====> rajouter des conditions au cas ou il n'y a pas de match dans l'historique ou s'il n'y en a qu'un
    if last_10_games.shape[0] == 0:
        return np.zeros((6))
    elif last_10_games.shape[0] == 1:
        last_10_games = last_10_games.apply(lambda x:   pd.Series([x['FTHG'], x['FTAG'], 0, x['HY'], x['HR'], 1])   if x['HomeTeam_id'] == club_id \
            else pd.Series([x['FTAG'], x['FTHG'], 1, x['AY'], x['AR'], 1]) \
            , axis = 1)
        return last_10_games.values()
    else:
        last_10_games = last_10_games.apply(lambda x:   pd.Series([x['FTHG'], x['FTAG'], 0, x['HY'], x['HR'], 1])   if x['HomeTeam_id'] == club_id \
            else pd.Series([x['FTAG'], x['FTHG'], 1, x['AY'], x['AR'], 1]) \
            , axis = 1)
        return last_10_games.mean().values

In [166]:
def create_game_history(home_id, away_id):

    last_5_games = DCU_games.loc[((DCU_games.HomeTeam_id == home_id) & (DCU_games.AwayTeam_id == away_id)) | ((DCU_games.HomeTeam_id == away_id) & (DCU_games.AwayTeam_id == home_id))  ,:]\
        .sort_values('Date', ascending = False).reset_index(drop = True).loc[1:11,:].drop(columns=['Date'])
    # voici les colonnes utilisées dans le modèle
    # ['H_goals', 'H_goals_conceded', 'H_HorA', 'H_yellow_cards', 'H_red_cards', 'A_yellow_cards', 'A_red_cards', 'exists']
    # Si home_id est l'équipe à domicile, H_HorA = 0 sinon 1
    if last_5_games.shape[0] == 0:
        return np.zeros((8))
    elif last_5_games.shape[0] == 1:
        last_5_games = last_5_games.apply(lambda x:   pd.Series([x['FTHG'], x['FTAG'], 0, x['HY'], x['HR'], x['AY'], x['AR'], 1])   if x['HomeTeam_id'] == home_id \
            else pd.Series([x['FTAG'], x['FTHG'], 1, x['AY'], x['AR'], x['HY'], x['HR'], 1]) \
            , axis = 1)
        return last_5_games.values

    else:
        last_5_games = last_5_games.apply(lambda x:   pd.Series([x['FTHG'], x['FTAG'], 0, x['HY'], x['HR'], x['AY'], x['AR'], 1])   if x['HomeTeam_id'] == home_id \
            else pd.Series([x['FTAG'], x['FTHG'], 1, x['AY'], x['AR'], x['HY'], x['HR'], 1]) \
            , axis = 1)
        return last_5_games.mean().values


In [183]:
def create_predict(home_club_id, away_club_id):
    # This is the model input
    # [home_input,away_input,game_input,player_input,best_player_input]
    return [create_club_history(home_club_id), create_club_history(away_club_id), create_game_history(home_club_id, away_club_id),\
        np.concatenate(( create_AVG_stats_array(home_club_id), create_AVG_stats_array(away_club_id) )),\
             np.concatenate(( create_MAX_stats_array(home_club_id), create_MAX_stats_array(away_club_id) ))]

In [185]:
len(create_predict(374, 38))

5

In [177]:
model_load = tf.keras.models.load_model('data/models/hagpb_dense_only')


In [208]:
DCU_games.tail(20)

,Date,Time,HomeTeam,AwayTeam,HomeTeam_id,AwayTeam_id,FTHG,FTAG,HY,AY,HR,AR,season,Div,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST,HF,AF,HC,AC,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,PSCH,PSCD,PSCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
858,2022-01-16,19:45,Marseille,Lille,6.0,43.0,1,1,1,5,0,1,21/22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
859,2022-01-16,14:00,Strasbourg,Montpellier,13.0,17.0,3,1,2,2,0,1,21/22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
860,2022-01-16,14:00,Reims,Metz,211.0,20.0,0,1,4,4,0,0,21/22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
861,2022-01-16,14:00,Monaco,Clermont,25.0,242.0,4,0,0,1,0,0,21/22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
862,2022-01-16,14:00,Lorient,Angers,11.0,374.0,0,0,3,1,0,1,21/22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
863,2022-01-16,12:00,Rennes,Bordeaux,14.0,18.0,6,0,1,5,0,1,21/22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
864,2022-01-19,18:00,Clermont,Strasbourg,242.0,13.0,0,2,2,0,0,0,21/22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
865,2022-01-19,18:00,Lille,Lorient,43.0,11.0,3,1,0,0,0,0,21/22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [206]:
game = DCU_games.iloc[-1,:]
X = create_predict(game['HomeTeam_id'], game['AwayTeam_id'])
model_load.predict([x.reshape(1,-1) for x in X])

array([[0.5974968 , 0.2427363 , 0.15976693]], dtype=float32)

In [187]:
res = create_predict(374, 38)
[x.shape for x in res]

[(6,), (6,), (8,), (192,), (144,)]

In [190]:
res[0].reshape((1,-1)).shape

(1, 6)